What is Milvus?

Milvus is an open-source vector database designed to store and search large volumes of high-dimensional vector embeddings efficiently.

Use case: Imagine you have 1 million text documents, and you’ve converted each into a 384-dimensional vector. Milvus helps you:
	•	Store all those vectors.
	•	Search which stored document is most similar to a new query vector — fast and at scale.

Why Milvus?
	•	Optimized for similarity search (cosine, inner product, Euclidean).
	•	Scalable to billions of vectors.
	•	Integrates well with embedding models (like OpenAI, Hugging Face, SentenceTransformer).
	•	Supports indexes like IVF, HNSW, and Flat for fast retrieval.
	•	Has APIs in Python, REST, etc.

In [2]:
from pymilvus import connections

connections.connect(
    uri=cluster_endpoint,
    token=api_key,
    secure=True
)

print("✅ Connected to Milvus Cloud!")

✅ Connected to Milvus Cloud!


In [3]:
docs = [
    "Walmart allows electronics returns within 30 days with a receipt.",
    "Suppliers must complete onboarding forms before shipment approval.",
    "Walmart Plus members get free shipping on all orders.",
    "Returns on perishable items are not accepted beyond 7 days.",
    "Employees can access HR policies via the internal portal."
]

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs)
print(embeddings)

Loads a pre-trained Sentence Transformer model:

This model converts sentences into dense vectors (embeddings).

all-MiniLM-L6-v2 is a lightweight, fast, and effective model for text similarity and semantic search.

Embeddings are numerical representations of text, typically 384-dimensional vectors for this model.

docs is a list of strings (e.g., sentences, paragraphs, or documents).

Each string gets converted into a vector.

The output embeddings is a NumPy array of shape [num_docs, 384].

In [ ]:
pip install sentence-transformers pymilvus langchain

In [ ]:
pip install "milvus-lite==2.4.1"  # for Walmart Element Platform

In [5]:
import sqlite3

In [27]:
from pymilvus import connections

# Use local storage (.db file)

connections.connect(alias="default2", uri="walmart_data2.db")

print("Connected to Milvus Cloud is Successful ")

Connected to Milvus Cloud is Successful 


In [28]:
docs = [
    "Walmart allows electronics returns within 50 days with a receipt.",
    "Suppliers must complete onboarding forms before shipment approval.",
    "Walmart Plus members get free shipping on all orders.",
    "Returns on perishable items are not accepted beyond 7 days.",
    "Employees can access HR policies via the internal portal."
]

In [5]:
docs = [
    "LTIMindtree employees must submit timesheets weekly through the internal portal.",
    "Clients are required to sign the Statement of Work (SOW) before project initiation.",
    "LTIMindtree provides free upskilling courses on AI and Cloud for all associates.",
    "Leave requests must be approved by the reporting manager at least 3 days in advance.",
    "Project teams can access compliance and security policies via the LTIMindtree intranet."
]

In [6]:
[doc for doc in docs]

['LTIMindtree employees must submit timesheets weekly through the internal portal.',
 'Clients are required to sign the Statement of Work (SOW) before project initiation.',
 'LTIMindtree provides free upskilling courses on AI and Cloud for all associates.',
 'Leave requests must be approved by the reporting manager at least 3 days in advance.',
 'Project teams can access compliance and security policies via the LTIMindtree intranet.']

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs)
print(embeddings)

[[-0.03970195  0.0017197  -0.06133857 ...  0.05272065  0.10862722
   0.00723266]
 [-0.04248329  0.05166248 -0.04707661 ...  0.03269951  0.01580414
   0.00862307]
 [-0.06850354 -0.04223825  0.00804254 ... -0.03274461 -0.00976729
  -0.03541669]
 [ 0.01848623  0.01614    -0.0055839  ...  0.03670225 -0.02595269
   0.00541953]
 [-0.04755552  0.02677119 -0.05872783 ...  0.03248743  0.06940335
  -0.04215242]]


In [13]:
print(embeddings)

[[-0.02364307  0.04532686  0.09156336 ... -0.0599112   0.02255651
   0.00900443]
 [-0.03858036 -0.05658962  0.01043053 ...  0.04141787  0.13493302
  -0.0421605 ]
 [-0.02691722 -0.05574032  0.09135328 ... -0.08896992  0.06445432
   0.0134097 ]
 [-0.05788542  0.02268525  0.06491429 ... -0.08462054 -0.03600617
   0.00783663]
 [-0.04330171  0.0290013  -0.08033062 ...  0.03523532  0.08481064
   0.03786615]]


SentenceTransformer :Converts text to embeddings (vectors)

embeddings = model.encode(docs)

Transforms a list of documents to numerical vector format

Milvus :A database that stores those vectors and enables fast similarity search
Use case: Semantic search, recommendation systems, chatbots, deduplication, etc.

In [8]:
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
]
schema = CollectionSchema(fields)
collection = Collection("LTIMindtree_docs", schema)

In [9]:
insert_data = [
    [doc for doc in docs],
    [vec.tolist() for vec in embeddings]
]
collection.insert(insert_data)

(insert count: 5, delete count: 0, upsert count: 0, timestamp: 460717463136894982, success count: 5, err count: 0, cost: 3)

In [10]:
collection.create_index(
    field_name="embedding",
    index_params={"index_type": "AUTOINDEX", "metric_type": "COSINE"}
)
collection.load()

In [11]:
query = "What is LTIMIndtree Leave Policy?"
query_embedding = model.encode([query]).tolist()

results = collection.search(
    data=query_embedding,
    anns_field="embedding",
    param={"metric_type": "COSINE", "params": {"nprobe": 10}},
    limit=1,
    output_fields=["content"]
)

for hit in results[0]:
    print("💡 Top Match:", hit.entity.get("content"), "→ Score:", hit.distance)

💡 Top Match: Leave requests must be approved by the reporting manager at least 3 days in advance. → Score: 0.4324192404747009


In [33]:
collection.create_index(
    field_name="embedding",
    index_params={"index_type": "AUTOINDEX", "metric_type": "COSINE"}
)
collection.load()

In [2]:
from sentence_transformers import SentenceTransformer

# Load pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample Walmart content (e.g., product or policy document)
docs = [
    "Return policy: Electronics must be returned within 45 days.",
    "Product: ₹999 kids backpack with cartoon print and zip pockets.",
    "Order placed after 8 PM will ship next day from Pune warehouse."
]

# Generate embeddings
embeddings = model.encode(docs)
print(embeddings)

[[-0.0151267   0.06271887  0.11397745 ... -0.03558155 -0.03983023
  -0.04348071]
 [-0.06026062  0.08771145 -0.06173638 ... -0.06180822 -0.00445606
   0.03151754]
 [-0.01856996 -0.02781065  0.0148665  ... -0.03499598 -0.02098577
   0.01342641]]


collection.create_index(...)

It creates a vector index on the field named "embedding" in the collection. This index makes similarity search faster.

Parameters Explained:

field_name="embedding"
	•	Refers to the vector field you stored (e.g., OpenAI or BERT embeddings).
	•	The field must be of type FLOAT_VECTOR.

index_params={"index_type": "AUTOINDEX", "metric_type": "COSINE"}''

index_type: "AUTOINDEX"
	•	You don’t manually pick IVF_FLAT, HNSW, etc.
	•	Milvus automatically selects the best index based on:
	•	Vector dimension
	•	Data size
	•	Search speed vs accuracy

Ideal for beginners or dynamic pipelines. You let Milvus decide the optimal ANN strategy.

metric_type: "COSINE"
	•	Specifies how to measure vector similarity.
	•	COSINE is used when you care about direction (not magnitude).
	•	Other options: "L2" (Euclidean), "IP" (Inner Product).

In LLM and NLP use cases like RAG, COSINE similarity is commonly used.

⸻
Summary: What this does?

Creates an auto-optimized index for fast similarity search on embeddings using cosine similarity.

Step 1: Define the User Query

query = "What is the return policy for electronics?"

You are simulating a natural language question, just like a Walmart employee or customer might ask.

⸻
 Step 2: Convert Query to Embedding

query_embedding = model.encode([query]).tolist()

🔍 What’s happening here:
	•	model.encode(...) converts the query to a vector embedding using a model like:
	•	sentence-transformers (e.g., all-MiniLM-L6-v2)
	•	OpenAI (text-embedding-ada-002) or Azure OpenAI


data=query_embedding	The vector you’re searching with (your query embedding)
anns_field="embedding"	Name of the field in your collection where document vectors are stored

param={"metric_type": "COSINE", "params": {"nprobe": 10}}	

Use cosine similarity. 
nprobe controls search depth – higher = better recall

limit=1	Return the top 1 most similar result
output_fields=["content"]	Also return the actual document content (text), not just vector match info

for hit in results[0]:
    print("💡 Top Match:", hit.entity.get("content"), "→ Score:", hit.distance)

🔍 What this does:
	•	results[0] is the list of matches for your single query.
	•	hit.entity.get("content") gets the actual text chunk (e.g., policy paragraph).
	•	hit.distance is the similarity score (lower is better for L2, higher is better for cosine).

📌 Example output:

💡 Top Match: "Walmart allows returns of electronics within 30 days of purchase with a receipt." → Score: 0.91
Lower L2 distance = More similar
	•	Higher L2 distance = Less similar
